In [ ]:
import requests
import json
import os

# --- Configuration ---
# It's best practice to load API keys from environment variables.
# For demonstration, a placeholder is included. Replace it or set the variable.
API_KEY = os.getenv('ASI_ONE_API_KEY')
BASE_URL = "https://api.asi1.ai/v1"



headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# --- Tool Definition ---
# This is the actual Python function that gets executed.
# In a real-world scenario, this would make a request to a weather API.
def get_weather(latitude, longitude):
    """
    Simulates fetching weather data for a given latitude and longitude.
    """
    print(f"\n---> Executing get_weather(latitude={latitude}, longitude={longitude})")
    weather_data = {
        "temperature": 15,
        "unit": "celsius",
        "description": "Partly cloudy with a slight breeze."
    }
    # The tool's output must be a string.
    return json.dumps(weather_data)

# --- Tool Schema for the Model ---
# This is the JSON schema that describes the tool to the language model.
# It tells the model the tool's name, purpose, and required parameters.
get_weather_tool_schema = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get the current temperature for a specific location using its latitude and longitude.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "number",
                    "description": "The latitude of the location, e.g., 51.5074"
                },
                "longitude": {
                    "type": "number",
                    "description": "The longitude of the location, e.g., -0.1278"
                }
            },
            "required": ["latitude", "longitude"]
        }
    }
}


def run_conversation():
    """
    Manages the conversation flow, including making API calls and executing tools.
    """

    messages = [
        {
            "role": "system",
            "content": "You are a helpful weather assistant. When a user asks for the weather, use the get_weather tool to find the information. Do not make up weather data."
        },
        {
            "role": "user",
            "content": "What's the current weather in London (latitude 51.5074 and longitude -0.1278)?"
        }
    ]

    print("--- 1. Sending initial request to the model ---")

    payload = {
        "model": "asi1-mini",
        "messages": messages,
        "tools": [get_weather_tool_schema],
        "tool_choice": "auto" # 'auto' lets the model decide whether to use a tool.
    }

    try:
        response = requests.post(f"{BASE_URL}/chat/completions", headers=headers, json=payload)
        response.raise_for_status() 
        response_json = response.json()
        
        print("\n--- 2. Received response from the model ---")
        print(json.dumps(response_json, indent=2))

        response_message = response_json["choices"][0]["message"]

        
        if not response_message.get("tool_calls"):
            print("\n--- Model responded directly without using a tool. ---")
            print(f"Assistant: {response_message['content']}")
            return

        print("\n--- 3. Model decided to use a tool. ---")
        
        messages.append(response_message)
        
       
        tool_calls = response_message["tool_calls"]
        for tool_call in tool_calls:
            function_name = tool_call["function"]["name"]
            
            if function_name == "get_weather":
                
                function_args = json.loads(tool_call["function"]["arguments"])
                
               
                function_response = get_weather(
                    latitude=function_args.get("latitude"),
                    longitude=function_args.get("longitude")
                )
                
                print(f"<--- Tool execution returned: {function_response}")

               
                messages.append({
                    "tool_call_id": tool_call["id"],
                    "role": "tool",
                    "name": function_name,
                    "content": function_response
                })

 
        print("\n--- 4. Sending tool output back to the model for a final answer ---")
        second_payload = {
            "model": "asi1-mini",
            "messages": messages
        }
        second_response = requests.post(f"{BASE_URL}/chat/completions", headers=headers, json=second_payload)
        second_response.raise_for_status()
        
        final_response_json = second_response.json()
        final_answer = final_response_json["choices"][0]["message"]["content"]
        
        print("\n--- 5. Received final, user-facing response ---")
        print(f"Assistant: {final_answer}")

    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        print(f"Response Body: {e.response.text}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# Run the main conversation function
if __name__ == "__main__":
    run_conversation()


--- 1. Sending initial request to the model ---

--- 2. Received response from the model ---
{
  "id": "id_liEx0lB7XwtiISuPX",
  "model": "asi1-mini",
  "executable_data": [],
  "intermediate_steps": [],
  "conversation_id": null,
  "thought": [
    "\n\n"
  ],
  "choices": [
    {
      "index": 0,
      "finish_reason": "tool_calls",
      "message": {
        "role": "assistant",
        "content": "",
        "reasoning": "\n\n",
        "tool_calls": [
          {
            "id": "call_xRTac",
            "index": 0,
            "type": "function",
            "function": {
              "name": "get_weather",
              "arguments": "{\"longitude\":-0.1278,\"latitude\":51.5074}"
            }
          }
        ]
      }
    }
  ],
  "usage": {
    "prompt_tokens": 140,
    "completion_tokens": 8,
    "total_tokens": 148
  }
}

--- 3. Model decided to use a tool. ---

---> Executing get_weather(latitude=51.5074, longitude=-0.1278)
<--- Tool execution returned: {"temperature